Data available to download here:
https://opendata.cbs.nl/portal.html?_la=en&_catalog=CBS&tableId=00372eng&_theme=994

In [21]:
import pandas as pd

CBS_PATH = "CBS_dutch_power.csv"
mc3toTWh = 102.36 #divide by 102.36 to convert mlnc3 to TWh

In [22]:
df = pd.read_csv('CBS.csv',sep=';')

df['year'] = df['Periods'].astype('str').str[0:4]
df['period_type'] = df['Periods'].astype('str').str[4:6]
df['month'] = df['Periods'].astype('str').str[6:8]

df_monthly = df[df['period_type'] == 'MM'].reset_index().copy()

df_monthly = df_monthly[['ElectricityPowerPlants_22','year','month']].copy()
df_monthly['ElectricityPowerPlants_22']=df_monthly['ElectricityPowerPlants_22'].astype('int')

df_2019 = df_monthly[df_monthly['year'] == '2019'].copy()
df_2020 = df_monthly[df_monthly['year'] == '2020'].copy()
df_2021 = df_monthly[df_monthly['year'] == '2021'].copy()
df_2022 = df_monthly[(df_monthly['year'] == '2022') & (df_monthly['month'].isin(['01','02','03','04','05','06']))].copy()

df_output = pd.concat([df_2019,df_2020,df_2021,df_2022])
df_output['country'] = 'NL'
df_output['type'] = 'power'
df_output['source'] = 'CBS'
df_output.rename(columns={'ElectricityPowerPlants_22':'demand'},inplace=True)
df_output['demand'] = df_output['demand']/mc3toTWh
df_output.to_csv(CBS_PATH, index=False)

In [10]:
df22 = df_monthly[['TotalConsumption_20',
                    'ElectricityPowerPlants_22','month']].loc['2022'].groupby('month').mean()/mc3toTWh
df22.columns=['Total','Power']
df22['Industry&Households']=df22['Total']-df22['Power']
df22 = df22.round(2)
df22.to_csv('NL_22.csv')